In [1]:
import re
import json
import pandas as pd

In [6]:
with open('../data/raw/properties_dict.json', 'r') as f:
    properties_dict = json.load(f)
# data = [(d['postcode'], d['price']) for d in properties_dict.values()]
# data = pd.DataFrame(data, columns=['postcode', 'price'])

In [7]:
data = pd.DataFrame(properties_dict)
data = data.T

In [12]:
data.head()

,price,address,postcode,bedrooms,baths,parking,property_type,property_features
https://www.domain.com.au/cp2008-11-rose-lane-melbourne-vic-3000-15806114?topspot=1,$60,CP2008/11 Rose Lane Melbourne VIC 3000,3000,− 0 Beds,− 0 Baths,1 Parking,Carspace,[]
https://www.domain.com.au/unit-1713-200-spencer-st-melbourne-vic-3000-16069309,$440 per week,Unit 1713/200 Spencer St Melbourne VIC 3000,3000,1 Bed,1 Bath,1 Parking,Apartment / Unit / Flat,[]
https://www.domain.com.au/701-101-therry-st-melbourne-vic-3000-16069017,NEW corner apartment Keep your bond,701/101 Therry St Melbourne VIC 3000,3000,1 Bed,1 Bath,− Parking,Apartment / Unit / Flat,"[Air conditioning, Gym, Intercom, Indoor Spa, ..."
https://www.domain.com.au/816-101-therry-st-melbourne-vic-3000-16068338,New apartment No bond needed Pets welcome,816/101 Therry St Melbourne VIC 3000,3000,2 Beds,2 Baths,− Parking,Apartment / Unit / Flat,"[Ensuite(s), Air conditioning, Ensuite, Indoor..."
https://www.domain.com.au/3902-220-spencer-street-melbourne-vic-3000-16067459,$650 per week,3902/220 Spencer Street Melbourne VIC 3000,3000,2 Beds,2 Baths,1 Parking,Apartment / Unit / Flat,[Furnished]


In [13]:

def process_price(text):
    price = re.findall(r'@?\$\d+(?:.\d+)?', text.replace(',', ''))
    if len(price) == 0:
        price = re.findall(r'@?\d+(?:.\d+)?', text.replace(',', ''))
    if len(price) > 0:
        out = price[0]
        if out[0] == '$':
            out = float(out[1:])
        else:
            try:
                out = float(out)
            except ValueError:
                out = ''
    else:
        out = ''
    return out

def get_freq(text):
    text = text.lower()
    if re.search(r"week[ly]?|wk|p/?w", text) is not None:
        out = 'week'
    elif re.search(r"month[ly]?|p/?m", text) is not None:
        out = 'month'
    # elif 'annum' in text or ' pa' in text:
    elif re.search(r"\bannum|p\.?a\.?\b|annually|year", text) is not None:
        out = 'year'
    else:
        out = 'week'
    return out

data['freq'] = data.price.apply(get_freq)
data['processed_price'] = data.price.apply(process_price)
data = data[data.processed_price != '']
data.loc[data.freq == 'unk', 'freq'] = 'week'

data.loc[data.freq == 'month', 'processed_price'] = data.loc[data.freq == 'month', 'processed_price'] * 12 / 52
data.loc[data.freq == 'year', 'processed_price'] = data.loc[data.freq == 'year', 'processed_price'] / 52

/var/folders/6h/1_h0dg6s6wj8yz6rsq27r0dr0000gn/T/ipykernel_31031/3452211303.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[data.freq == 'unk', 'freq'] = 'week'
/var/folders/6h/1_h0dg6s6wj8yz6rsq27r0dr0000gn/T/ipykernel_31031/3452211303.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[data.freq == 'month', 'processed_price'] = data.loc[data.freq == 'month', 'processed_price'] * 12 / 52
/var/folders/6h/1_h0dg6s6wj8yz6rsq27r0dr0000gn/T/ipykernel_31031/3452211303.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentatio

In [17]:
def transform_baths(text):
    baths = re.findall(r'(\d{1,2})', text)
    if len(baths) != 0:
        out = int(baths[0])
    else:
        out = 0
    return out

data['baths'] = data['baths'].apply(transform_baths)

/var/folders/6h/1_h0dg6s6wj8yz6rsq27r0dr0000gn/T/ipykernel_31031/2734197672.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['baths'] = data['baths'].apply(transform_baths)


In [18]:
def transform_bedrooms(text):
    bedrooms = re.findall(r'(\d{1,2})', text)
    if len(bedrooms) != 0:
        out = int(bedrooms[0])
    else:
        out = 0
    return out

data['bedrooms'] = data['bedrooms'].apply(transform_bedrooms)

/var/folders/6h/1_h0dg6s6wj8yz6rsq27r0dr0000gn/T/ipykernel_31031/1489931775.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['bedrooms'] = data['bedrooms'].apply(transform_bedrooms)


In [20]:
def transform_parking(text):
    parking = re.findall(r'(\d{1,2})', text)
    if len(parking) != 0:
        out = int(parking[0])
    else:
        out = 0
    return out

data['parking'] = data['parking'].apply(transform_parking)


/var/folders/6h/1_h0dg6s6wj8yz6rsq27r0dr0000gn/T/ipykernel_31031/2357379544.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['parking'] = data['parking'].apply(transform_parking)


In [23]:
data.drop(columns='price', inplace=True)

,address,postcode,bedrooms,baths,parking,property_type,property_features,freq,processed_price
https://www.domain.com.au/cp2008-11-rose-lane-melbourne-vic-3000-15806114?topspot=1,CP2008/11 Rose Lane Melbourne VIC 3000,3000,0,0,1,Carspace,[],week,60.0
https://www.domain.com.au/unit-1713-200-spencer-st-melbourne-vic-3000-16069309,Unit 1713/200 Spencer St Melbourne VIC 3000,3000,1,1,1,Apartment / Unit / Flat,[],week,440.0
https://www.domain.com.au/3902-220-spencer-street-melbourne-vic-3000-16067459,3902/220 Spencer Street Melbourne VIC 3000,3000,2,2,1,Apartment / Unit / Flat,[Furnished],week,650.0
https://www.domain.com.au/3405-288-spencer-street-melbourne-vic-3000-16067117,3405/288 Spencer Street Melbourne VIC 3000,3000,1,1,0,Apartment / Unit / Flat,"[INSIDE:, Split System Heating, Air conditioni...",week,440.0
https://www.domain.com.au/1007-238-flinders-st-melbourne-vic-3000-16066003,1007/238 Flinders St Melbourne VIC 3000,3000,1,1,0,Apartment / Unit / Flat,[],week,320.0
...,...,...,...,...,...,...,...,...,...
https://www.domain.com.au/2-34a-freda-street-inverloch-vic-3996-15996474,2/34a Freda Street Inverloch VIC 3996,3996,3,2,2,House,"[Fireplace(s)*, Heating*]",week,600.0
https://www.domain.com.au/28-beachcomber-drive-inverloch-vic-3996-15920343,28 Beachcomber Drive Inverloch VIC 3996,3996,4,2,0,House,"[Gas*, Internal Laundry*, Secure Parking*, Fur...",week,550.0
https://www.domain.com.au/14-inverloch-parade-inverloch-vic-3996-16040296,14 Inverloch Parade Inverloch VIC 3996,3996,2,1,0,House,"[Internal Laundry*, Heating*, Garden, Split Sy...",week,420.0
https://www.domain.com.au/10-hopetoun-street-inverloch-vic-3996-15935242,10 Hopetoun Street Inverloch VIC 3996,3996,2,1,1,House,"[Heating*, Balcony / Deck*]",week,370.0


In [226]:
data.to_csv('../data/curated/processed_data.csv')